In [1]:
from re import L
from sklearn import metrics
import rank_metric as metrics
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_extraction.text import TfidfVectorizer
import xml.etree.ElementTree as ET
import matplotlib as plt
import pandas as pd
import numpy as np
import pprint as pp
import tarfile
from sklearn.feature_extraction.text import CountVectorizer
import math
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from bertviz import model_view, head_view
import torch
from transformers import *
import pprint

In [2]:


model_path = 'bert-base-uncased'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device
tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path,  output_hidden_states=True, output_attentions=True)  
model = AutoModel.from_pretrained(model_path, config=config).to(device)
# model = AutoModel.from_pretrained(model_path, config=config).to(device)

loading configuration file config.json from cache at /Users/anteklasik/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at /Users/anteklasik/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/v

## Data Loading


In [2]:
#parsing real patient cases
import pickle
import os
trials = "data/qrels-clinical_trials.txt"
trials = pd.read_csv(trials, sep='\t', names=["query_id", "dummy", "docid", "rel"])
trials = trials.drop("dummy", axis = 1)
trials_to_use = set(trials['docid'])
if not os.path.exists("data/doc_brief_title.bin") or not os.path.exists("data/doc_ids.bin") or not os.path.exists("data/docs_detailed_description.bin") or not os.path.exists("data/docs_brief_summary.bin") or not os.path.exists("data/docs_criteria.bin"):
    #parsing clinical trials
    
    tar = tarfile.open("data/clinicaltrials.gov-16_dec_2015.tgz", "r:gz")
    clinical_trials = {}
    docs_brief_title = {}
    docs_detailed_description={}
    docs_brief_summary={}
    docs_criteria={}
    ids_brief_title = []
    ids_detailed_description=[]
    ids_brief_summary=[]
    ids_criteria=[]

    i = 0
    for element in tar:
        if element.size > 500:
            txt = tar.extractfile(element).read().decode("utf-8", "strict")
            root = ET.fromstring(txt)
            for doc_id in root.iter('nct_id'):
                temp_id = doc_id.text
            if temp_id not in trials_to_use:
                continue
            i+=1
            for brief_title in root.iter('brief_title'):
                id_title=doc_id.text
                docs_brief_title[temp_id]=brief_title.text
                ids_brief_title.append(temp_id)

            
            
            for brief_summary in root.iter('brief_summary'):
                for child in brief_summary:
                    docs_brief_summary[temp_id]=child.text.strip()
                    # ids_brief_summary.append(temp_id)
                    id_sum=doc_id.text
            if temp_id!=id_sum:
                
                docs_brief_summary[temp_id]=docs_brief_title[temp_id]


            for detailed_description in root.iter('detailed_description'):
                for child in detailed_description:
                    docs_detailed_description[temp_id]=child.text.strip()
                    # ids_detailed_description.append(temp_id)
                    id_detailed=doc_id.text

            if temp_id!=id_detailed:
                
                docs_detailed_description[temp_id]=docs_brief_title[temp_id]+docs_brief_summary[temp_id]


        

            for criteria in root.iter('criteria'):
                for child in criteria:
                    id_crit=doc_id.text
                    doc=child.text.strip().split("Exclusion")
                    doc=doc[0].split()
                    temp=[]
                    for x in doc:
                        if x=="-" or x=="Inclusion" or x=="Criteria:":
                            continue
                        temp.append(x)
                    doc=' '.join(temp)
                    if len(doc)==0:
                        docs_criteria[temp_id]=docs_detailed_description[temp_id]
                    else:
                        docs_criteria[temp_id]=doc

                    # ids_criteria.append(temp_id)
            if temp_id!=id_crit:
                
                docs_criteria[temp_id]=docs_detailed_description[temp_id]

                    
            
    print(f"Number of trials to use: {i}")

    pickle.dump(trials,open("data/trials.bin","wb"))
    pickle.dump(docs_brief_title, open( "data/docs_brief_title.bin", "wb" ) )
    pickle.dump(ids_brief_title, open( "data/doc_ids.bin", "wb" ) )
    pickle.dump(docs_brief_summary,open( "data/docs_brief_summary.bin", "wb" ))
    pickle.dump(docs_detailed_description,open( "data/docs_detailed_description.bin", "wb" ))
    pickle.dump(docs_criteria,open( "data/docs_criteria.bin", "wb" ))
    pickle.dump(ids_brief_summary, open( "data/doc_ids_summary.bin", "wb" ) )
    pickle.dump(ids_detailed_description, open( "data/doc_ids_description.bin", "wb" ) )
    pickle.dump(ids_criteria, open( "data/doc_ids_criteria.bin", "wb" ) )
else:
    trials=pickle.load(open("data/trials.bin","rb"))
    docs_brief_title = pickle.load( open( "data/docs_brief_title.bin", "rb" ) )
    ids_brief_title = pickle.load( open( "data/doc_ids.bin", "rb" ) )
    docs_detailed_description=pickle.load( open( "data/docs_detailed_description.bin", "rb" ) )
    docs_brief_summary=pickle.load( open( "data/docs_brief_summary.bin", "rb" ) )
    docs_criteria=pickle.load( open( "data/docs_criteria.bin", "rb" ) )
    # ids_detailed_description=pickle.load( open( "data/doc_ids_description.bin", "rb" ) )
    # ids_brief_summary=pickle.load( open( "data/doc_ids_summary.bin", "rb" ) )
    # ids_criteria=pickle.load( open( "data/doc_ids_criteria.bin", "rb" ) )

Number of trials to use: 3626


In [5]:

dict_detailed=docs_detailed_description
docs_brief_summary=list(docs_brief_summary.values())
docs_brief_title=list(docs_brief_title.values())
docs_detailed_description=list(docs_detailed_description.values())
docs_criteria=list(docs_criteria.values())
print(len(docs_brief_summary),type(docs_brief_summary))
print(len(docs_brief_title),type(docs_brief_title))
print(len(docs_detailed_description),type(docs_detailed_description))
print(len(docs_criteria),type(docs_criteria))
print(len(trials),type(trials))

    





3626 <class 'list'>
3626 <class 'list'>
3626 <class 'list'>
3626 <class 'list'>
3870 <class 'pandas.core.frame.DataFrame'>


### Loading the queries 

In [6]:
Queries = "data/topics-2014_2015-summary.topics"
with open(Queries, 'r') as queries_reader:
    txt = queries_reader.read()
root = ET.fromstring(txt)
cases = {}
for q in root.iter('TOP'):
    q_number = q.find("NUM").text
    q_title = q.find('TITLE').text
    cases[q_number] = q_title

### Class for functions used by Language Model with Jelineck-Mercer smooting and Vector Space Model



In [8]:
class NLP:
    def __init__(self, eval_table: pd.DataFrame, cases: dict, ngram_range, docs_brief_title, ids):
        # self.docs_brief_title = documents
        self.eval_table = eval_table
        self.docs_brief_title = docs_brief_title
        self.ids = ids
        self.cases = cases
        self.vsm = TfidfVectorizer(ngram_range=ngram_range, analyzer='word', stop_words = None)
        self.X = self.vsm.fit_transform(docs_brief_title)
        self.vectorizer = CountVectorizer(ngram_range=(1,1), analyzer='word', stop_words = None)
        self.Y=self.vectorizer.fit_transform(docs_brief_title)

    def test_vsm(self, query):
        tfidf = self.vsm.transform([query])
        score = 1 - pairwise_distances(self.X, tfidf, metric='cosine')
        for x in range(len(score)):
            score[x]=float(score[x])
        results = pd.DataFrame(list(zip(self.ids, score)), columns=['_id', 'score'])
        ordered_results = results.sort_values(by=['score'], ascending=False).reset_index(drop=True)
        return ordered_results,results

    def eval(self, result: pd.DataFrame, id):
        def_sim = self.eval_table[self.eval_table["query_id"] == int(id)]
        def_sim = def_sim[def_sim["rel"] != 0]
        rel_ids = def_sim["docid"]
        num_of_rel = def_sim["rel"].count()
        num_of_results = result["_id"].count()

        if num_of_rel == 0:
            return [0, 0, 0, 0, 0]

        #precision@10
        top10 = result['_id'][:10]
        p10 = np.intersect1d(top10, rel_ids).size / 10

        #recall
        recall = np.size(np.intersect1d(result["_id"][:100], rel_ids)) / num_of_rel
        rel_res_vector = np.zeros((num_of_results,))
        for index, row in def_sim.iterrows():
            rel_res_vector = rel_res_vector + ((result['_id'] == row.docid)*row.rel).to_numpy()
        #ndcg5
        ndcg5 = metrics.ndcg_at_k(r = rel_res_vector, k = 5, method = 1)
        ap = metrics.average_precision(rel_res_vector, num_of_rel)
        mrr = metrics.mean_reciprocal_rank(rel_res_vector)

        return [p10, 1 - recall, ndcg5, ap, mrr]

    def evalPR(self, scores, query_id):

        aux = self.eval_table.loc[self.eval_table['query_id'] == int(query_id)]
        idx_rel_docs_brief_title = aux.loc[aux['rel'] != (0)]

        [dummyA, rank_rel, dummyB] = np.intersect1d(scores['_id'], idx_rel_docs_brief_title['docid'], return_indices=True)
        rank_rel = np.sort(rank_rel) + 1
        total_relv_ret = rank_rel.shape[0]
        if total_relv_ret == 0:
            return [np.zeros(11, ), [], total_relv_ret]

        recall = np.arange(1, total_relv_ret + 1) / idx_rel_docs_brief_title.shape[0]
        precision = np.arange(1, total_relv_ret + 1) / rank_rel

        precision_interpolated = np.maximum.accumulate(precision)
        recall_11point = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
        precision_11point = np.interp(recall_11point, recall, precision)

        if False:
            print(total_relv_ret)
            print(rank_rel)
            print(recall)
            print(precision)
            plt.plot(recall, precision, color='b', alpha=1)  # Raw precision-recall
            plt.plot(recall, precision_interpolated, color='r', alpha=1)  # Interpolated precision-recall
            plt.plot(recall_11point, precision_11point, color='g', alpha=1)  # 11-point interpolated precision-recall

        return [precision_11point, recall_11point, total_relv_ret]


    def words_and_indexes_association(self,query):
        q_words=[]
        word_idx=[]
        all_words=self.vectorizer.get_feature_names()
        for x in range(len(query)):
            for y in range(len(all_words)):
                if query[x] == all_words[y]:
                    q_words.append(query[x])
                    word_idx.append(y)
                    break
        return word_idx

    def JMS(self):
        x_array=self.Y.toarray()
        x_array=np.array(x_array)
        #sum of rows
        p1=np.sum(x_array,axis=0)
        #sum of cols
        p2=np.sum(x_array,axis=1)
        #number of words in whole corpus
        p3=np.sum(x_array)
        #probability of term in corpus
        matrix_corpus=np.divide(p1,p3)
        #probability of term in document
        p2=np.reshape(p2,(p2.size,1))
        matrix_documents=np.divide(x_array,p2)  
        return matrix_corpus,matrix_documents

    def query_formating(self,query):
        query=query[:-11]
        for y in query:
                if y in ".,":
                    query=query.replace(y,'')
                
        return query 

    def prob_with_JMS(self,index_word,index_doc,lamb,matrix_corp,matrix_doc):
        prob=0
        for x in index_word:
            #print(x,index_doc)
        
            p_md=matrix_doc[index_doc][x]
            p_mc=matrix_corp[x]
            prob+=math.log(lamb*p_md+(1-lamb)*p_mc)
        return prob

    def JSM_test(self,query,lamb,matrix_corpus,matrix_documents):
        scores_now=[]
        q=query.split()
        word_idx=self.words_and_indexes_association(q)
        #print(len(self.ids))
        for x in range(len(self.ids)):
            #print(word_idx,x,lamb)
            prob=self.prob_with_JMS(word_idx,x,lamb,matrix_corpus,matrix_documents)
            scores_now.append(prob)
        results = pd.DataFrame(list(zip(self.ids, scores_now)), columns=['_id', 'score'])
        ordered_results = results.sort_values(by=['score'], ascending=False).reset_index(drop=True)
        return ordered_results,results
    



# Loading the dataset split

In [7]:

cases_training=pickle.load(open("data/cases_training.bin","rb"))
cases_test=pickle.load(open("data/cases_test.bin","rb"))



In [10]:
    
def extract_cls(pairs,query_pairs, embeddings, batch_size=32):
    

    # Iterate over all documents, in batches of size <batch_size>
    for batch_idx in range(0, len(query_pairs), batch_size):
        

        # Get the current batch of samples
        batch_data = query_pairs[batch_idx:batch_idx + batch_size]

        inputs = tokenizer.batch_encode_plus(batch_data, 
                                       return_tensors='pt',  # pytorch tensors
                                       add_special_tokens=True,  # Add CLS and SEP tokens
                                       max_length = 512, # Max sequence length
                                       truncation = True, # Truncate if sequences exceed the Max Sequence length
                                       padding = True) # Add padding to forward sequences with different lengths
        
        # Forward the batch of (query, doc) sequences
        with torch.no_grad():
            inputs.to(device)
            outputs = model(**inputs)

        # Get the CLS embeddings for each pair query, document
        batch_cls = outputs['hidden_states'][-1][:,0,:]
        
        # L2-Normalize CLS embeddings. Embeddings norm will be 1.
        batch_cls = torch.nn.functional.normalize(batch_cls, p=2, dim=1)
        
        # Store the extracted CLS embeddings from the batch on the memory-mapped ndarray
        embeddings[batch_idx:batch_idx + batch_size] = batch_cls.cpu()
        
        
    embeddings=pd.DataFrame(embeddings)
    embeddings["pairs"]=pairs
    
        
    return embeddings

In [12]:
def query_pairs(cases_training):
    dictio={}
    
    h=0
    #add column pair in which we have tuple (document id, query id)
    
    trials=pickle.load(open("data/trials.bin","rb"))
    for x in trials.iterrows():
        dictio[str(x[1][0])]=[]
    for x in trials.iterrows():
      
        # for each row i qrels file dict[query].append all of the documents that are labelled
        
        dictio[str(x[1][0])].append(str(x[1][1]))
    k=[]  
    
    for x in cases_training.items():
        scores=pd.DataFrame()
        
        #if query is qrels file
        if x[0] in dictio:
            temp=dictio[x[0]]
            
            
            
           
            l=[]
            for z in ids_brief_title:
                 l.append((z,str(x[0])))
            
            scores["pairs"]=l
            #add column with field name+vsm or jms ending with corresponding scores
            #for each row in these results find these that are labelled in qrels file and put them in relscores dataframe
            for z in scores.iterrows():
                
                
                if z[1][0][0] in temp:
                    # print(z[1][0][0])
                    # h+=1
                    # print(h)

                    k.append(z[1][0])
    
    
    return k

In [13]:
def pairs(cases_training):
    k=query_pairs(cases_training)
    l=[]
    # print(k)
    myNLP=NLP(trials, cases_training, (1,1), docs_detailed_description, ids_brief_title)
    for x in k:
        l.append((myNLP.query_formating(cases_training[x[1]]),dict_detailed[x[0]]))
    return l,k

In [14]:
def all_pairs(cases):
    k=[]
    l=[]
    for x in cases.items():

        for y in ids_brief_title:
            k.append((y,x[0]))
    myNLP=NLP(trials, cases, (1,1), docs_detailed_description, ids_brief_title)
    for x in k:
        l.append((myNLP.query_formating(cases[x[1]]),dict_detailed[x[0]]))
    return l,k

    
    

# Computing all of the scores for labelled pair (query, document)

# Matrix Y definition

In [16]:
#based on matrix y (final results dataframe) we make a matrix y in which we have binary labels for each 
# (query, document) pair from the previous matrix
def matrix_y_def(final_scores):
    print(type(final_scores))
    list_of_relevances=[]
    binary_relevance=[]
    # final_scores=pickle.load(open("data/final_relevant_scores.bin","rb"))
    trials=pickle.load(open("data/trials.bin","rb"))

    for f,y in final_scores.iterrows():
            for x in trials.iterrows():
                #here we use the pairs column to find the corresponding label in qrels file
                # if (x[1][1],str(x[1][0]))==y[1][0]:
                # print(x[1][1],str(x[1][0]))
                # print(y["pairs"])
                if (x[1][1],str(x[1][0]))==y["pairs"]:
                    
                    z=x[1][2]
                    #if the label is equal to 2 we change it to 1 
                    list_of_relevances.append(z) 
                    if z>0:
                        z=1
                        
                    
                    binary_relevance.append(z)          
    # print(len(list_of_relevances))
    pickle.dump(list_of_relevances,open("data/matrix_y.bin","wb"))
    return list_of_relevances,binary_relevance


# Matrix X normalization 

# Logistic Regression 
## using X and Y matrices

In [18]:
def log_regr_train(matrix_x,c,typ,matrix_y,rel):
    #using both matrices with logistic regression
    # matrix_y=pickle.load(open("data/matrix_y.bin","rb"))
   
    if typ=="a":
        clf = LogisticRegression(C=c,class_weight=None).fit(matrix_x, matrix_y, sample_weight=None)
    if typ=="b":
        clf = LogisticRegression(C=c,class_weight="balanced").fit(matrix_x, matrix_y, sample_weight=None)
    if typ=="c":
        clf = LogisticRegression(C=c,class_weight=None).fit(matrix_x, matrix_y, sample_weight=rel)
    #coeficients correspond to weight we should give each column in matrix X
    # (how important are the scores obtaned from each document field)
    return clf.coef_

# Computing the scores using logistic regression weights

In [22]:
def calc_scores_embed(coef,cases,c):
    records=[]
    
    prec = np.zeros(11,)
    test = np.zeros(5,)
    myNLP=NLP(trials, cases, (1,1), docs_detailed_description, ids_brief_title)
    for x in cases.items():
        scores=pd.DataFrame()
        result=pd.DataFrame()
        statistics_embed={}
        doc_list=[]
        zeros=np.zeros(len(ids_brief_title))
        z={}
        z[x[0]]=x[1]

        l,k=all_pairs(z)
        for y in k:
            doc_list.append(y[0])

        matrix_x=np.zeros((len(l), 768))

        matrix_x=extract_cls(k,l,matrix_x,batch_size=32)
        # print(matrix_x["pairs"])
        del matrix_x["pairs"]
        result=matrix_x*coef[0]

        scores["score"]=result.sum(axis=1)
        
        # scores["pairs"]=j
        # print(scores)
        
        scores["_id"]=doc_list
        # print(scores)
        scores=scores.sort_values(by=["score"],ascending=False).reset_index(drop=True)
        # print(scores)
        p10, recall, ndcg5, ap, mrr = myNLP.eval(scores, x[0])
        test +=np.array([p10, ndcg5, mrr, ap, recall])

        prec_11point, recal_11point, tot_relv_ret = myNLP.evalPR(scores, x[0])
        prec +=prec_11point
    prec = prec / (len(cases.keys()))
    test = test / (len(cases.keys()))
    for p, r in zip(prec, recal_11point):
        records.append((c, p, r))
    statistics_embed["C="+str(c)]=test
    stats=pd.DataFrame.from_dict(statistics_embed, orient="index", columns=["p10", "ndcg5", "mrr", "ap", "recall"])
    return stats,records


# Tuning the regularization parameter

# A)

In [25]:
ids_training_cases=list(cases_training.keys())
final_stats=pd.DataFrame()
for c in [0.1,0.3,0.5,0.7,0.9]:
    kf = KFold(3, shuffle=True, random_state=23)
    stats=np.zeros((1,5))
    for id_train,id_valid in kf.split(ids_training_cases,ids_training_cases):
        
        
        training_set={}
        temp=[]
        validation_set={}
        for id in id_train:
            temp=list(cases_training.keys())
            training_set[temp[id]] = cases_training[temp[id]]
        for id in id_valid:
            id=int(id)
            validation_set[temp[id]] = cases_training[temp[id]]
        # print("set")
        # print(len(training_set))
        # print("pary")
        l,k=pairs(training_set)
        # print(len(l),len(k))


        matrix_x=np.zeros((len(l), 768))
        # print("zaczął tworzyć x")
        matrix_x=extract_cls(k,l,matrix_x,batch_size=32)

        # print("zaczął tworzyć y")
        matrix_y,binary=matrix_y_def(matrix_x)
        # matrix_x=matrix_x_norm()
        # print(len(matrix_x))
        # print(len(matrix_y))
        # print(matrix_x)
        
        del matrix_x["pairs"]
        coef=log_regr_train(matrix_x,c,"a",binary,matrix_y)
        pickle.dump(matrix_x,open("data/matirex.bin","wb"))
        pickle.dump(coef,open("data/coef.bin","wb"))
        pickle.dump(c,open("data/c.bin","wb"))
        
        print("Stats")
        # temp,records=calc_scores_embed(coef,validation_set,[0.15,0.25,0.4,0.15],c)
        temp,records=calc_scores_embed(coef,validation_set,c)
        stats+=temp
        # print("_____________")
    print("Stats for c=",c)
    stats=stats/3
    final_stats=pd.concat([final_stats,stats])
    print(stats)  
pickle.dump(final_stats,open("data/debug_embed_a","wb"))       



zaczął tworzyć x
zaczął tworzyć y
<class 'pandas.core.frame.DataFrame'>
Stats
3626


In [ ]:
l,k=pairs(cases_training)
matrix_x=np.zeros((len(l), 768))
matrix_x=extract_cls(k,l,matrix_x,batch_size=32)
pickle.dump(matrix_x,open("data/embeddings_final","wb"))  
# matrix_x=pickle.load(open("data/embeddings_final","rb")) 
# matrix_y=pickle.load(open("data/embed_matrix_y","rb")) 
# binary=pickle.load(open("data/embed_binary","rb")) 
matrix_y,binary=matrix_y_def(matrix_x)

print(len(matrix_x))
print(len(matrix_y))
if len(matrix_x)==len(matrix_y):
    del matrix_x["pairs"]
    print("usunal pary")


coef=log_regr_train(matrix_x,5.9,"a",binary,matrix_y)


stats,records=temp=calc_scores_embed(coef,cases_test,5.9)
pickle.dump(stats,open("data/letor_a_embed_stats","wb")) 
pickle.dump(records,open("data/letor_a_embed_records","wb")) 
print(stats)